# Getting training data

This notebook contains code to download metadata and transcripts for videos contained in training data (`Data/normalized_data/train.csv`) as well as in the encountered videos with metadata (`Data/normalized_data/videos_metadata.csv`) and to process them into a pandas DataFrame. The code here is based on the code provided by [Papadamou et al.](https://github.com/kostantinos-papadamou/pseudoscience-paper/tree/main/youtubehelpers)

## Download videos' metadata and transcripts

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from socket import error as SocketError
import time
import glob
import subprocess
import pandas as pd
import os
import json

In [ ]:
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
YOUTUBE_API_KEY = 'YOUR_YOUTUBE_DATA_API_KEY'

In [ ]:
youtube_api = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=YOUTUBE_API_KEY)

In [ ]:
VIDEO_BASE_DIR = '../Data/raw_data/videos_data'

In [ ]:
def download_video_metadata(video_id):
    while True:
        try:
            # Send request to get video's information
            response = youtube_api.videos().list(
                part='id,snippet,contentDetails,statistics',
                id=video_id
            ).execute()

            # Get Video Details
            try:
                return response['items'][0]
            except:
                return None

        except (HttpError, SocketError) as error:
            print(f'--- HTTP Error occurred while retrieving information for VideoID: {video_id}. [ERROR]: {error}')
            time.sleep(30)


def is_video_transcript_downloaded(video_id):
    video_transcript = glob.glob(f'{VIDEO_BASE_DIR}/{video_id}/{video_id}_transcript.*')
    if len(video_transcript) > 0:
        return True
    return False


def download_video_transcript(video_id):
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}_transcript'

    try:
        # download_video_transcript.sh needs to have execute permissions
        output = subprocess.check_output(f'bash download_video_transcript.sh {video_url} {path}', shell=True)
        if "HTTP_ERROR" in str(output):
            print(output)
            return
    except subprocess.CalledProcessError as e:
        pass
    return


def download_video(video_id, download_transcript=True):
    print('Downloading metadata...')
    video_metadata = download_video_metadata(video_id)
    
    if video_metadata is None:
        print(f'ERROR: Video Metadata not available for video: {video_id}')
        return None

    if download_transcript and not is_video_transcript_downloaded(video_id):
        print('Downloading transcript...')
        download_video_transcript(video_id)

    return video_metadata

## Train data

In [ ]:
videos = pd.read_csv('../Data/normalized_data/train.csv')

In [ ]:
youtube_ids = list(videos['youtube_id'])
len(youtube_ids)

In [ ]:
if not os.path.exists(VIDEO_BASE_DIR):
    os.mkdir(VIDEO_BASE_DIR)

In [ ]:
for video_id in youtube_ids:
    path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}.json'
    if not os.path.exists(path):
        print('Downloading video', video_id)
        video_details = download_video(video_id)
        with open(path, 'w') as f:
            json.dump(video_details, f)
        print('Done')
        time.sleep(10)

### Process downloaded train data

In [ ]:
from datetime import datetime
import numpy as np
import csv

In [ ]:
def process_train_video_metadata(path):
    with open(path, 'r') as f:
        video = json.load(f)

        return {
            'youtube_id': video['id'],
            'published_at': video['snippet']['publishedAt'],
            'updated_at': datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
            'view_count': video['statistics']['viewCount'] if 'viewCount' in video['statistics'] else np.nan,
            'like_count': video['statistics']['likeCount'] if 'likeCount' in video['statistics'] else np.nan,
            'dislike_count': video['statistics']['dislikeCount'] if 'dislikeCount' in video['statistics'] else np.nan,
            'favourite_count': video['statistics']['favoriteCount'] if 'favoriteCount' in video['statistics'] else np.nan,
            'comment_count': video['statistics']['commentCount'] if 'commentCount' in video['statistics'] else np.nan,
            'duration': video['contentDetails']['duration']
        }

def get_video_transcript(path):
    transcript_path = glob.glob(path)
    if len(transcript_path) == 0:
        return ''
    
    transcript_path = transcript_path[0]
    with open(transcript_path, 'r') as f:
        return f.read()

In [ ]:
processed_train_videos = []

for video_id in youtube_ids:
    path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}.json'
    transcript_path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}_transcript.*'
    if os.path.exists(path):
        processed_video = process_train_video_metadata(path)
        processed_video['transcript'] = get_video_transcript(transcript_path)
        processed_video['annotation'] = videos[videos['youtube_id'] == video_id]['annotation'].values[0]
        
        processed_train_videos.append(processed_video)

In [ ]:
processed_train_videos_df = pd.DataFrame(processed_train_videos)

In [ ]:
processed_train_videos_df.info()

In [ ]:
processed_train_videos_df.head()

In [ ]:
processed_train_videos_df.to_csv('../Data/normalized_data/train_processed.csv', index=False, quoting=csv.QUOTE_ALL)

## Encountered data with metadata

In [ ]:
videos = pd.read_csv('../Data/normalized_data/videos_metadata.csv')

In [ ]:
youtube_ids = list(videos['youtube_id'])
len(youtube_ids)

In [ ]:
if not os.path.exists(VIDEO_BASE_DIR):
    os.mkdir(VIDEO_BASE_DIR)

In [ ]:
for video_id in youtube_ids:
    path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}.json'
    if not os.path.exists(path):
        print('Downloading video', video_id)
        video_details = download_video(video_id, download_transcript=False)
        os.mkdir(os.path.join(VIDEO_BASE_DIR, video_id))
        with open(path, 'w') as f:
            json.dump(video_details, f)
        print('Done')
        time.sleep(10)

### Process downloaded train data

In [ ]:
from datetime import datetime
import numpy as np
import csv

In [ ]:
def process_encountered_video_metadata(path, videos_df):
    with open(path, 'r') as f:
        video = json.load(f)

        video_metadata = videos_df.loc[videos_df['youtube_id'] == video['id'], ['duration_seconds', 'duration_minutes', 'duration_hours', 'encountered_home', 'encountered_search', 'encountered_recommend', 'encountered_all']]

        return {
            'youtube_id': video['id'],
            'published_at': video['snippet']['publishedAt'],
            'title': video['snippet']['title'],
            'description': video['snippet']['description'],
            'channel_id': video['snippet']['channelId'],
            'language': video['snippet']['defaultAudioLanguage'] if 'defaultAudioLanguage' in video['snippet'] else np.nan,
            'duration_seconds': video_metadata['duration_seconds'].values[0],
            'duration_minutes': video_metadata['duration_minutes'].values[0],
            'duration_hours': video_metadata['duration_hours'].values[0],
            'view_count': video['statistics']['viewCount'] if 'viewCount' in video['statistics'] else np.nan,
            'like_count': video['statistics']['likeCount'] if 'likeCount' in video['statistics'] else np.nan,
            'dislike_count': video['statistics']['dislikeCount'] if 'dislikeCount' in video['statistics'] else np.nan,
            'favorite_count': video['statistics']['favoriteCount'] if 'favoriteCount' in video['statistics'] else np.nan,
            'comment_count': video['statistics']['commentCount'] if 'commentCount' in video['statistics'] else np.nan,
            'encountered_home': video_metadata['encountered_home'].values[0],
            'encountered_search': video_metadata['encountered_search'].values[0],
            'encountered_recommend': video_metadata['encountered_recommend'].values[0],
            'encountered_all': video_metadata['encountered_all'].values[0],
        }

In [ ]:
processed_encountered_videos = []

for video_id in youtube_ids:
    path = f'{VIDEO_BASE_DIR}/{video_id}/{video_id}.json'
    if os.path.exists(path):
        processed_video = process_encountered_video_metadata(path, videos)
        processed_encountered_videos.append(processed_video)

In [ ]:
processed_encountered_videos_df = pd.DataFrame(processed_encountered_videos)

In [ ]:
processed_encountered_videos_df.info()

In [ ]:
processed_encountered_videos_df.head()

In [ ]:
processed_encountered_videos_df.to_csv('../Data/normalized_data/videos_metadata_processed.csv', index=False, quoting=csv.QUOTE_ALL)